In [ ]:
import pickle
import pandas as pd
import numpy as np
from collections import defaultdict
from utility import strip_digt

def vt_contract_to_df(contracts):
    data = defaultdict(list)
    for contract in contracts:
        data['vt_symbol'].append(contract.vt_symbol)
        data['product'].append(contract.product.value)
        data['exchange'].append(contract.exchange.value)
        data['size'].append(contract.size)
        data['pricetick'].append(contract.pricetick)
    df = pd.DataFrame(data)
    return df

def get_mean_price(commodity):
    df = pd.read_csv('source_data/future_mean_price.csv', index_col=0, header=None, names=['mean_close'])
    d = dict(df['mean_close'])
    return d.get(commodity.upper(), None)

with open('source_data/contracts.data', 'rb') as f:
    data = pickle.load(f)
    
    
df = vt_contract_to_df(data)
df['commodity'] = df['vt_symbol'].map(strip_digt)
df['product'] = df['product'].map({'期货': 'future'})
df = df[df['product'] == 'future'].copy()
df.drop_duplicates(subset='commodity', inplace=True)

df['mean_close'] = df['commodity'].map(get_mean_price)
df['backtest_capital'] = np.floor(df['mean_close'] * df['size'] / 1000) * 1000

export_items = ['commodity', 'product', 'exchange', 'size', 'pricetick', 'mean_close', 'backtest_capital']
df.to_csv('future_basic_data.csv', index=False, columns=export_items)

In [ ]:
future_basic_data = pd.read_csv('source_data/future_basic_data.csv', index_col=0)